# Basic Data Cleaning Before Filtering

This notebook is created to perform data cleaning on the real estate market trends in Conneticut. 

The raw data file was obtained from https://catalog.data.gov/dataset/real-estate-sales-2001-2018. On the website, the file is described to include:

`serial number`: Serial Number

`list year`: Year the property was listed for sale

`date recorded`: Date the sale was recorded locally

`town`: Town name

`address`: Address

`assessed value`: Value of the property used for local tax assesment

>With the exception of certain classified land, the assessment of each parcel of real property represents 70% of its estimated fair market value as of the date of a revaluation (Chapter 203 - Sec. 12-62, Sec. 12-62a and Sec. 12-63)

`sale amount`: Amount the property was sold for

`sales ratio`: Ratio of the sale price to the assessed value

`property type`: Type of property including: Residential, Commercial, Industrial, Apartments, Vacant, etc

`residential type`: Indicates whether property is single or multifamily residential

`Non Use Code`: Non usable sale code typically means the sale price is not reliable for use in the determination of a property value. See attachments in the dataset description page for a listing of codes

`Assessor Remarks`: Remarks from the  municipal assessor

`OPM remarks`: Remarks from OPM, Office of Policy and Management

`Location`: Lat / lon coordinates


Frequently used libraries are imported:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re

In [2]:
#Dataset imported
real_estate = pd.read_csv('../Datasets/Real_Estate_Sales_2001-2020_GL.csv')
#Reset Index
real_estate.reset_index(drop=True)

/var/folders/kv/4xjj0g696_lcyc7ms5pdd2t00000gn/T/ipykernel_17299/2421691908.py:2: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  real_estate = pd.read_csv('../Datasets/Real_Estate_Sales_2001-2020_GL.csv')


,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463000,Commercial,NaN,NaN,NaN,NaN,NaN
1,20002,2020,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,430000.0,0.588300,Residential,Single Family,NaN,NaN,NaN,NaN
2,200212,2020,03/09/2021,Avon,5 CHESTNUT DRIVE,130400.0,179900.0,0.724800,Residential,Condo,NaN,NaN,NaN,NaN
3,200243,2020,04/13/2021,Avon,111 NORTHINGTON DRIVE,619290.0,890000.0,0.695800,Residential,Single Family,NaN,NaN,NaN,NaN
4,200377,2020,07/02/2021,Avon,70 FAR HILLS DRIVE,862330.0,1447500.0,0.595700,Residential,Single Family,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997208,190272,2019,06/24/2020,New London,4 BISHOP CT,60410.0,53100.0,1.137665,Single Family,Single Family,14 - Foreclosure,NaN,NaN,NaN
997209,190284,2019,11/27/2019,Waterbury,126 PERKINS AVE,68280.0,76000.0,0.898400,Single Family,Single Family,25 - Other,PRIVATE SALE,NaN,NaN
997210,190129,2019,04/27/2020,Windsor Locks,19 HATHAWAY ST,121450.0,210000.0,0.578300,Single Family,Single Family,NaN,NaN,NaN,NaN
997211,190504,2019,06/03/2020,Middletown,8 BYSTREK DR,203360.0,280000.0,0.726300,Single Family,Single Family,NaN,NaN,NaN,NaN


Make a subset of the dataframe that contains unit information to test a function that will get rid of any unit indicaiton from the address column:

In [3]:
contains_unit_or_hash_or_dash = real_estate[real_estate["Address"].str.contains("UNIT|suite|apt|apartment|\#|-|Ste", case = False) ==True]
contains_unit_or_hash_or_dash 

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
20,20000179,2020,02/22/2021,Brookfield,72 HOMESTEAD LN,94770.0,170000.0,0.557400,Residential,Condo,NaN,NaN,NaN,NaN
28,200024,2020,11/09/2020,Columbia,WEBSTER LANE,60300.0,75000.0,0.804000,Vacant Land,NaN,NaN,NaN,NaN,NaN
38,200938,2020,05/13/2021,Danbury,15-17 BOUGHTON ST #1-5,118900.0,230000.0,0.516900,Residential,Condo,NaN,I14129-5,NaN,NaN
47,200242,2020,01/27/2021,East Haven,130 COE AVE #51,28240.0,73000.0,0.386849,Residential,Condo,NaN,NaN,GOOD SALE PER MLS,NaN
49,200375,2020,04/06/2021,East Haven,55 THOMPSON ST #5A,76960.0,140000.0,0.549700,Residential,Condo,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997168,1900695,2019,02/24/2020,Stamford,39 WOODWAY ROAD UNIT C6,188410.0,379000.0,0.497124,Condo,Condo,07 - Change in Property,NaN,TOTAL RENOVATION PER MLS - SEE PREVIOUS SALE #...,POINT (-73.51447 41.10279)
997184,19040,2019,11/07/2019,Monroe,53 HARVESTER RD,223100.0,310000.0,0.719700,Single Family,Single Family,NaN,NaN,NaN,NaN
997186,190812,2019,07/06/2020,West Hartford,115 STEELE ROAD,344120.0,505000.0,0.681400,Single Family,Single Family,NaN,NaN,NaN,NaN
997188,190195,2019,12/06/2019,West Hartford,7 WEBSTER HILL BOULEVARD,172830.0,270000.0,0.640100,Single Family,Single Family,NaN,NaN,NaN,NaN


Make a function that will clean the address column by removing the unit information:

In [4]:
def unit_number_remover(address):
    """
    Remove the unit number from the given address using a regular expression.
    
    Args:
        address (str or None): The address string.
        
    Returns:
        str or None: Street Address without unit number.
    """
    address_without_units = ""
    
    if address is not None:
        address = str(address)
        #remove any stand-alone unit number
        address_without_units = re.sub(r'(?:(?:UNIT|suite|apt|apartment|\#|-|Ste)\s*[a-zA-Z0-9/-]+)?', '', address, flags=re.IGNORECASE)

        #remove a unit number starting with an alphabetic letter after a hyphen and the street number
        address_without_units = re.sub(r'-(?=[A-Za-z])[\w\d]*', '', address_without_units)

        #remove a unit number that precedes a street number and a hyphen
        address_without_units = re.sub(r'\d+-', '', address_without_units)

        #title-case the address
        address_without_units = address_without_units.title()

        #place a comma between the street number and the street name
        address_without_units = re.sub(r'(\d+)\s+([A-Z][a-z]+)', r'\1, \2', address_without_units)

    return address_without_units

Check the `unit_number_remover` function:

In [327]:
contains_unit_or_hash_or_dash["Full Address"] = contains_unit_or_hash_or_dash["Address"].apply(unit_number_remover)
contains_unit_or_hash_or_dash[["Full Address", "Address"]].sample(10)

/var/folders/kv/4xjj0g696_lcyc7ms5pdd2t00000gn/T/ipykernel_57519/178112894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contains_unit_or_hash_or_dash["Full Address"] = contains_unit_or_hash_or_dash["Address"].apply(unit_number_remover)


,Full Address,Address
309994,"81, St",81 STEWART ST
775858,"310, Boston Post Rd",310 BOSTON POST RD #135
373054,"11, Cutler St",11-13 CUTLER ST
188113,"73, Park Ave 411",73-79 PARK AVE 411
719062,"26, Douglas Avenue",26 DOUGLAS AVENUE UNIT A
863930,"661, Silver Sands Rd",661 SILVER SANDS RD #A1
754488,"16, Seymour Road",16 SEYMOUR ROAD UNIT 20B
553720,"114, Park Ter",114-116 PARK TER
184403,"38, Brunswick Ave",38 BRUNSWICK AVE
389826,"45, Hope St",45-47 HOPE ST


As the function is effective applied in the subset of the dataset, apply the function to the original dataset:

In [5]:
real_estate["Full Address"] = real_estate["Address"].apply(unit_number_remover) 
real_estate[["Full Address", "Address"]].sample(10)

,Full Address,Address
886821,"450, Dexter Dr",450 DEXTER DR
987326,"46, Bellevue Ave",46 BELLEVUE AVE
277612,"1, Maplewood Dr",1 MAPLEWOOD DR
760447,"11, Haley Rd",11 HALEY RD
401132,"18, Condon Dr",18 CONDON DR
26750,"77, Oakland Ave",77 OAKLAND AVE
949777,"7, Lia Fail Way",7 LIA FAIL WAY
377640,"13, Joiners Rd",13 JOINERS RD
304578,"106, Lincoln Way",106 LINCOLN WAY
903497,"43, Corinne Drive",43 CORINNE DRIVE


Since the columnn `location` is in a string datatype in `POINT (XX.XXXXXX, XX.XXXXX)` format, it will be divided into two columns of the float datatype. 

First, use Regular Expression to find the two floats that might be negative or positive.

In [6]:
def extract_coordinates(location):
    if isinstance(location, str):
        match = re.search(r'\((-?\d+\.\d+)\s(-?\d+\.\d+)\)', location)
        if match:
            return match
    return None
    
def extract_latitude(coordinates):
    if coordinates is not None:
        return coordinates[1]
    return None

def extract_longitude(coordinates):
    if coordinates is not None:
        return coordinates[2]
    return None


##### Explanation of the regex search:
- `re.search`` matches the characters search literally (case sensitive)

This regex Capturing Group is repeated twice to find the coordinats in the string column of `location`: 
> \((-?\d+\.\d+)\s <

- `?` matches the previous token (which is `-`) between zero and one times, as many times as possible, giving back as needed (greedy). As a result, it preserves the minus sign, indicating that the number could be negative.
- `\d` matches a digit (equivalent to [0-9])
- `+` matches the previous token (which is a digit) between one and unlimited times, as many times as possible, giving back as needed (greedy). As a result, it preserves all the digits.
- `\.` matches the character which the period `.`
- `\d` matches a digit (equivalent to [0-9])
- `+` matches the previous token (which is a digit) between one and unlimited times, as many times as possible, giving back as needed (greedy)
- `\s` matches any whitespace character.

Reference: Used the built-in explanation on https://regex101.com.


##### Apply the coordinate extracting functions to the dataset:

In [7]:
real_estate['coordinates'] = real_estate['Location'].apply(extract_coordinates)
real_estate['latitude'] = real_estate['coordinates'].apply(extract_latitude)
real_estate['longitude'] = real_estate['coordinates'].apply(extract_longitude)
real_estate = real_estate.drop(['coordinates'], axis=1)
real_estate['latitude'] = real_estate['latitude'].astype(float)
real_estate['longitude'] = real_estate['longitude'].astype(float)

In [8]:
real_estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997213 entries, 0 to 997212
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Serial Number     997213 non-null  int64  
 1   List Year         997213 non-null  int64  
 2   Date Recorded     997211 non-null  object 
 3   Town              997213 non-null  object 
 4   Address           997162 non-null  object 
 5   Assessed Value    997213 non-null  float64
 6   Sale Amount       997213 non-null  float64
 7   Sales Ratio       997213 non-null  float64
 8   Property Type     614767 non-null  object 
 9   Residential Type  608904 non-null  object 
 10  Non Use Code      289681 non-null  object 
 11  Assessor Remarks  149864 non-null  object 
 12  OPM remarks       9934 non-null    object 
 13  Location          197697 non-null  object 
 14  Full Address      997213 non-null  object 
 15  latitude          197696 non-null  float64
 16  longitude         19

From this output, it is evident that `latitude` and `longitude` columns have the successfully extracted coordinates.

Now, get rid of the rows with incorrect values / information which was discovered in EDA (For more information, please refer to `EDA before Data Cleaning` jupyter notebook in the `EDA` folder:

In [9]:
search_string = "INCORRECT"

# Filter rows that contain the search string, including missing values
matching_rows = real_estate[real_estate['OPM remarks'].str.contains(search_string, case = False, na=False)]

matching_rows

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,Full Address,latitude,longitude
981,200562,2020,02/03/2021,Danbury,19 MILL RD,263600.0,4.150000e+05,0.635181,Residential,Single Family,25 - Other,AFFORDABLE HOUSING / B15001-20-19,INCORRECT DATA PER TOWN RECORDS,POINT (-73.53692 41.38822),"19, Mill Rd",-73.53692,41.38822
5006,200151,2020,12/21/2020,Guilford,33 INDIAN COVE RD,241200.0,9.200000e+05,0.262174,Residential,Single Family,07 - Change in Property,NaN,19x8 AREA OF TOWN SKETCH IS INCORRECT PER MLS ...,POINT (-72.68572 41.25516),"33, Indian Cove Rd",-72.68572,41.25516
13081,201166,2020,08/31/2021,Hartford,255 MARTIN ST,61226.0,2.050000e+05,0.298663,Residential,Single Family,25 - Other,NaN,INCORRECT ADDRESS - SEE SALE #201140,NaN,"255, Martin St",NaN,NaN
23728,200920,2020,05/10/2021,Danbury,42 BRENTWOOD CI,278800.0,5.300000e+05,0.526038,Residential,Condo,25 - Other,B16001-170,INCORRECT ADDRESS? - ALSO SEE #200710,POINT (-72.85037 41.44558),"42, Brentwood Ci",-72.85037,41.44558
25599,200471,2020,01/20/2021,Manchester,14 SUMMIT STREET,120400.0,8.850000e+04,1.360452,Residential,Two Family,25 - Other,NaN,TOWN SKETCH INCORRECT - REAR ENCLOSED PORCH IS...,NaN,"14, Summit Street",NaN,NaN
59835,20200091,2020,08/19/2021,Willington,456 TOLLAND TPKE,2238530.0,5.000000e+09,0.000448,Apartments,NaN,25 - Other,APARTMENTS-COMMERICAL,INCORRECT SALE PRICE,NaN,"456, Tolland Tpke",NaN,NaN
60894,20200078,2020,07/06/2021,Willington,224 RIVER ROAD,223070.0,3.187900e+08,0.000700,Residential,Single Family,25 - Other,COLONIAL,INCORRECT SALE PRICE - NO MLS,POINT (-72.30341 41.86603),"224, River Road",-72.30341,41.86603
65620,20200102,2020,09/28/2021,Willington,7 TOLLAND TURNPIKE ROAD,112040.0,1.941491e+08,0.000577,Residential,Three Family,25 - Other,MULTI FAMILY,INCORRECT SALE PRICE,NaN,"7, Tolland Turnpike Road",NaN,NaN
816742,160251,2016,12/08/2016,Waterbury,101 MADELINE AVE #16,776240.0,1.300000e+05,5.971077,Condo,Condo,25 - Other,NaN,ASSESSMENT INCORRECT,NaN,"101, Madeline Ave",NaN,NaN
830164,17350,2017,02/21/2018,Hartford,463 ALBANY AV,53760.0,6.350000e+04,0.846614,Single Family,Single Family,14 - Foreclosure,Foreclosure related sale,ADDRESS IS INCORRECT,NaN,"463, Albany Av",NaN,NaN


Filter rows for incorrect_real_estate where 'OPM remarks' do not contain the word 'incorrect' in upper or lower case:

In [10]:
correct_real_estate = real_estate[~real_estate['OPM remarks'].str.contains(search_string, case = False, na=False)]
correct_real_estate

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,Full Address,latitude,longitude
0,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463000,Commercial,NaN,NaN,NaN,NaN,NaN,"230, Wakelee Ave",NaN,NaN
1,20002,2020,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,430000.0,0.588300,Residential,Single Family,NaN,NaN,NaN,NaN,"390, Turnpike Rd",NaN,NaN
2,200212,2020,03/09/2021,Avon,5 CHESTNUT DRIVE,130400.0,179900.0,0.724800,Residential,Condo,NaN,NaN,NaN,NaN,"5, Chestnut Drive",NaN,NaN
3,200243,2020,04/13/2021,Avon,111 NORTHINGTON DRIVE,619290.0,890000.0,0.695800,Residential,Single Family,NaN,NaN,NaN,NaN,"111, Northington Drive",NaN,NaN
4,200377,2020,07/02/2021,Avon,70 FAR HILLS DRIVE,862330.0,1447500.0,0.595700,Residential,Single Family,NaN,NaN,NaN,NaN,"70, Far Hills Drive",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997208,190272,2019,06/24/2020,New London,4 BISHOP CT,60410.0,53100.0,1.137665,Single Family,Single Family,14 - Foreclosure,NaN,NaN,NaN,"4, Bishop Ct",NaN,NaN
997209,190284,2019,11/27/2019,Waterbury,126 PERKINS AVE,68280.0,76000.0,0.898400,Single Family,Single Family,25 - Other,PRIVATE SALE,NaN,NaN,"126, Perkins Ave",NaN,NaN
997210,190129,2019,04/27/2020,Windsor Locks,19 HATHAWAY ST,121450.0,210000.0,0.578300,Single Family,Single Family,NaN,NaN,NaN,NaN,"19, Hathaway St",NaN,NaN
997211,190504,2019,06/03/2020,Middletown,8 BYSTREK DR,203360.0,280000.0,0.726300,Single Family,Single Family,NaN,NaN,NaN,NaN,"8, Bystrek Dr",NaN,NaN


In [11]:
correct_real_estate.to_csv('../Datasets/correct_real_estate.csv')